## 🧾 Mainframe JCL to Structured JSON Parser

This notebook demonstrates a proof-of-concept that parses Mainframe components (`JCL`, `JCLPROC`, and `SQLLIB`) into a detailed, structured JSON format. For confidentiality reasons I am not mentioning the actual input JCL's and Output Json. 

## 🎯 Objective
Help application teams:
- Analyze job execution flow
- Identify database interactions
- Extract SQL logic & transformations
- Assist in modernization planning


In [ ]:
# Sample JCL input (pseudo)
JCL = """..."""  # Keep your original sample here

JCLPROC = """..."""  # Your sample PROC

SQLLIB = """..."""  # Your SQLLIB input


## 🧠 Structured Prompt for JCL → JCLPROC → SQL Extraction

This prompt guides an LLM to extract structured information from three mainframe components:

- **`JCL`**: Defines the main job, referencing procedures (`JCLPROC`)
- **`JCLPROC`**: Contains job steps and datasets, often pointing to `SQLLIB` members
- **`SQLLIB`**: Contains SQL statements — must be deeply parsed if referenced

---

### 🎯 Objective

Generate a single, structured **JSON object** that captures:

In [77]:
getjson4 = f"""
You are extracting structured information from Mainframe components: JCL, JCLPROC, and SQLLIB.

- JCL defines the main job and may invoke one or more procedures (JCLPROC).
- JCLPROC defines execution steps, often referencing SYSIN or DD statements that point to SQLLIB members.
- SQLLIB contains SQL statements and must be fully parsed if referenced.

---

Your task is to generate a single, comprehensive JSON that captures the full execution hierarchy and SQL lineage.

### JSON Hierarchy:
job → jcl_procs_called → procs_used → sqllib_used → sql_operations

---

### JSON Structure and Fields:

#### 1. `job`:
- `name`: Job name from the JCL
- `jcl_procs_called`: List of PROC names referenced via EXEC steps
- `procs_used`: List of detailed PROC objects

#### 2. For each `proc`:
- `proc_name`: Name of the PROC
- `input_files`: Only include datasets that are **PS (Physical Sequential)**, **GDG (Generation Data Groups)**, or **PDS ending in `.SQLLIB`** if used by the proc  
  ❗ Do **not** include:
  - other PDS (Partitioned Datasets) not ending with `.SQLLIB`
- `output_files`: Same rules as above — only include PS or GDG datasets
- `sqllib_used`: List of referenced SQLLIB members  
  - For each member:
    - `member_name`: Name of the SQLLIB member (e.g., `VX20101D`)
    - `sql_operations`: List of SQL statements with the following fields:
      - `operation`: SQL command type (e.g., INSERT, DELETE, SELECT)  
        ❗️ **Do not include `COLLECT STATISTICS`**
      - `query`: Full SQL text
      - `impacted_columns`: List of columns affected by the operation
      - `where_clause`: WHERE condition if present
      - `group_by`: GROUP BY clause if present
      - `order_by`: ORDER BY clause if present
      - `transformations`: List of expressions involving functions, concatenations, calculations, or logic applied to columns  
        - For each transformation:
          - `expression`: The full transformed expression (e.g., `XX.ABC_DT || '00:00:00'`)
          - `columns_involved`: List of columns used in the expression (e.g., `XX.ABC_DT`)
          - `source_table`: The actual table (not alias) the column comes from
      - `joins`: List of joins with:
        - `left_table`: Use the **actual table name**, not an alias
        - `right_table`: Use the **actual table name**, not an alias
        - `on_clause`: The full join condition
        - `join_type`: e.g., INNER, LEFT
        - ❗ Ensure both `left_table` and `right_table` match those in `tables_involved`
      - `unions`: If UNION/UNION ALL is used, include subqueries
      - `subqueries`: Any SELECTs used within WHERE or FROM clauses, with the involved tables
      - `tables_involved`: List of all tables referenced in the operation, each with:
        - `table_name`
        - `access_type`: e.g., SELECT, INSERT, DELETE

---

### Constraints:
- ❌ Do **not** include datasets from unrelated PDS in `input_files` or `output_files`
- ❌ Do **not** omit `.SQLLIB` datasets if used — include them in `input_files`
- ❌ Do **not** include `COLLECT STATISTICS` operations in the final JSON
- ✅ Resolve all aliases used in joins or transformations to their corresponding base table names
- ✅ SQLLIB members must be fully parsed when referenced
- ✅ Return the result in **pure JSON format only** — no markdown, no extra text

---

### Input Content:
{JCL}
{JCLPROC}
{SQLLIB}
"""


In [12]:
# First let's do an import
from dotenv import load_dotenv

# Next it's time to load the API keys into environment variables
load_dotenv(override=True)

# Check the keys
import os
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set - please head to the troubleshooting guide in the setup folder")
    


OpenAI API Key exists and begins sk-proj-


In [13]:
from openai import OpenAI

# And now we'll create an instance of the OpenAI class
openai = OpenAI()

In [78]:
# Create a list of messages in the familiar OpenAI format

messages = [{"role": "user", "content": getjson4}]

#### OPENAI

In [ ]:
## Run the Prompt and Get JSON (Code Cell)

openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
response = response.choices[0].message.content
print(response)


#### GROQ

In [65]:
groq_api_key = os.getenv('GROQ_API_KEY')

In [ ]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
json_response = response.choices[0].message.content
print(json_response)

#### Conclusion : OPENAI accuracy is better